https://www.hackdeploy.com/tensorflow-word2vec-tutorial-from-scratch/

### Skip-Gram Model 
#### Predicting context words given center word

In [ ]:
import numpy as np
from collections import Counter
from collections import defaultdict
import tensorflow as tf
# import unittest

A bag-of-words is a representation of text that describes the occurrence of words within a document. It is called a “bag” of words, because any information about the order or structure of words in the document is discarded. The model is only concerned with whether known words occur in the document, not where in the document.

In [ ]:
# Dataset we are using


SENTENCES = [
             "machine learning engineers can build great data models",
             "the more data you have the better your model",
             "these predictions sound right, but it is all about your data",
             "your data can provide great value"
            ]

In [ ]:
# Bag of words function

def bag_of_words(input_sentence:list, delimiter=" ") -> dict:
    '''Converts the input_sentence to bag of words representation

    Parameters
    ----------
    input_sentence : iterable
        Text to be converted to bag of words
    delimiter : char
        Words in the text are separated by this character
    
    Returns
    -------
    output : dict
        Dictionary of words and their frequency
    '''
    output = Counter()
    # print(input_sentence[0].split(delimiter))
    for sentence in input_sentence:
            output.update(sentence.split(delimiter))

    return dict(output)


In [ ]:
# Unit Test
def test_bag_of_words():
    sentences = ["Deep learning is a Deep concept"]
    expected_output = {'Deep': 2, 'learning':1, 'is': 1, 'a': 1, 'concept': 1}
    received_output = bag_of_words(sentences)
    assert expected_output == received_output, f"\n Expected Output= {expected_output}\
                                             vs \n Returned Output= {received_output} \n"
    
test_bag_of_words()


In [ ]:
def generate_vocabulary():
    pass

def get_word_to_index(bag_of_words : dict):
    ''' Returns a mapping from word to its index in the vocab

    Parameters
    ----------
    bag_of_words : dict
        A frequency dictionary of words appearing in text

    Returns
    -------
    word_to_idx : dict
        A dictionary map from word to its index
    '''
    word_to_idx = {word:index  for index,word in enumerate(bag_of_words.keys())}
    return word_to_idx

def get_index_to_word(bag_of_words : dict):
    ''' Returns a mapping from index to corresponding word in the vocab

    Parameters
    ----------
    bag_of_words : dict
        A frequency dictionary of words appearing in text

    Returns
    -------
    idx_to_word : dict
        A dictionary map from index to corresponding word
    '''
    idx_to_word = {index:word  for index,word in enumerate(bag_of_words.keys())}
    return idx_to_word

In [ ]:
word_to_index = get_word_to_index(bag_of_words(SENTENCES))
index_to_word = get_index_to_word(bag_of_words(SENTENCES))
print(word_to_index,index_to_word)

{'machine': 0, 'learning': 1, 'engineers': 2, 'can': 3, 'build': 4, 'great': 5, 'data': 6, 'models': 7, 'the': 8, 'more': 9, 'you': 10, 'have': 11, 'better': 12, 'your': 13, 'model': 14, 'these': 15, 'predictions': 16, 'sound': 17, 'right,': 18, 'but': 19, 'it': 20, 'is': 21, 'all': 22, 'about': 23, 'provide': 24, 'value': 25} {0: 'machine', 1: 'learning', 2: 'engineers', 3: 'can', 4: 'build', 5: 'great', 6: 'data', 7: 'models', 8: 'the', 9: 'more', 10: 'you', 11: 'have', 12: 'better', 13: 'your', 14: 'model', 15: 'these', 16: 'predictions', 17: 'sound', 18: 'right,', 19: 'but', 20: 'it', 21: 'is', 22: 'all', 23: 'about', 24: 'provide', 25: 'value'}


In [ ]:
def generate_skip_gram_training_data(text_corpus,window_size):
    # Assuming text corpus as a list of strings.
    training_data = defaultdict(set)
    x = []
    y = []
    for sentence in text_corpus:
        words = sentence.split(" ")
        
        for i in range(len(words)):
            from_index = max(i-window_size,0)
            to_index = min(i+1+window_size,len(words))
            training_data[words[i]].update(set(words[from_index:to_index]))
            training_data[words[i]].remove(words[i])

    for center_word,context in training_data.items():
        for word in context:
            #train_data.append((center_word,word))
            x.append(center_word)
            y.append(word)

    words_dict = get_word_to_index(bag_of_words(text_corpus))
    x_indices = [words_dict[k] for k in x]
    y_indices = [words_dict[k] for k in y]
    x_indices = tf.convert_to_tensor(x_indices)
    y_indices = tf.convert_to_tensor(y_indices)
    x_indices = tf.keras.utils.to_categorical(x_indices)
    y_indices = tf.keras.utils.to_categorical(y_indices)
    train_data = tf.data.Dataset.from_tensor_slices((x_indices,y_indices))
    return train_data

In [ ]:
train_data = generate_skip_gram_training_data(SENTENCES,3)

In [ ]:
for element in train_data.as_numpy_iterator():
    print(index_to_word[np.argmax(element[0])])
    print(index_to_word[np.argmax(element[1])])
    print(f"len of vocabulary is {len(element[0])}")
    break

machine
can
len of vocabulary is 26


In [ ]:
def define_model():

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(64,activation=tf.nn.relu,input_shape=(26,)))
    model.add(tf.keras.layers.Dense(10,activation=tf.nn.relu)) # We will get embeddings of 10 dimesnsions
    model.add(tf.keras.layers.Dense(26,activation=tf.nn.softmax))

    return model

In [ ]:
model = define_model()

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')

In [ ]:
test_dataset = train_data.take(34) 
train_dataset = train_data.skip(34)

In [ ]:
test_dataset = test_dataset.batch(5)

In [ ]:
train_dataset = train_dataset.batch(5)

In [ ]:
model.fit(train_dataset,epochs=100)

Epoch 1/100
22/22 [==============================] - 0s 1ms/step - loss: 1.7869 - accuracy: 0.0364
Epoch 2/100
22/22 [==============================] - 0s 994us/step - loss: 1.7868 - accuracy: 0.0818
Epoch 3/100
22/22 [==============================] - 0s 954us/step - loss: 1.7869 - accuracy: 0.0364
Epoch 4/100
22/22 [==============================] - 0s 1ms/step - loss: 1.7868 - accuracy: 0.0818
Epoch 5/100
22/22 [==============================] - 0s 1ms/step - loss: 1.7869 - accuracy: 0.0364
Epoch 6/100
22/22 [==============================] - 0s 1ms/step - loss: 1.7869 - accuracy: 0.0818
Epoch 7/100
22/22 [==============================] - 0s 1ms/step - loss: 1.7869 - accuracy: 0.0364
Epoch 8/100
22/22 [==============================] - 0s 1ms/step - loss: 1.7869 - accuracy: 0.0818
Epoch 9/100
22/22 [==============================] - 0s 1ms/step - loss: 1.7869 - accuracy: 0.0455
Epoch 10/100
22/22 [==============================] - 0s 1ms/step - loss: 1.7868 - accuracy: 0.0818
Epoch

In [ ]:
model.evaluate(test_dataset)

7/7 [==============================] - 0s 2ms/step - loss: 11.7989 - accuracy: 0.0000e+00


[11.79892635345459, 0.0]

In [ ]:
embedding = model.layers[1]

In [ ]:
list(train_dataset.as_numpy_iterator())[0][0].shape

(26,)

In [ ]:
intermediate_layer_model = tf.keras.Model(inputs=model.input,
                                       outputs=model.layers[1].output)
intermediate_output = intermediate_layer_model(list(train_dataset.as_numpy_iterator())[0][0])

In [ ]:
intermediate_output

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[4.434657  , 1.2535803 , 5.280507  , 0.        , 0.        ,
        3.9689    , 0.        , 0.88672835, 6.474742  , 9.228638  ],
       [4.434657  , 1.2535803 , 5.280507  , 0.        , 0.        ,
        3.9689    , 0.        , 0.88672835, 6.474742  , 9.228638  ],
       [4.434657  , 1.2535803 , 5.280507  , 0.        , 0.        ,
        3.9689    , 0.        , 0.88672835, 6.474742  , 9.228638  ],
       [4.434657  , 1.2535803 , 5.280507  , 0.        , 0.        ,
        3.9689    , 0.        , 0.88672835, 6.474742  , 9.228638  ],
       [4.434657  , 1.2535803 , 5.280507  , 0.        , 0.        ,
        3.9689    , 0.        , 0.88672835, 6.474742  , 9.228638  ]],
      dtype=float32)>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=dcd03e1a-e120-4eeb-b05f-6684779c3d8f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>